In [1]:
import pandas as pd
from utilities.functions import *

# load the lookup table
geoid_lu_df = pd.read_csv('tbl/NCRPlaces_Census_04192024.csv')

# to test, limit the lookup table to a few locations to test each geography type
# we include an individual county, incorporated place, census designated place, 
# and a 1-to-many entry (Eagle River) that has multiple census tracts per single GVV ID
geoid_lu_df_test = geoid_lu_df[geoid_lu_df['name'].isin(['Eagle River', 'Fairbanks', 'Arctic Village', 'Fort Yukon'])]

In [2]:
geoid_lu_df_test

,id,name,alt_name,region,country,latitude,longitude,type,GEOIDFQ,PLACENAME,AREATYPE,COMMENT
10,AK124,Fairbanks,NaN,Alaska,US,64.8378,-147.716,community,0500000US02090,Fairbanks North Star Borough,County,Data represent information from nearest [COLUM...
45,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000201,Census Tract 2.01,Census tract,Data for this place represent multiple merged ...
46,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000202,Census Tract 2.02,Census tract,Data for this place represent multiple merged ...
47,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000204,Census Tract 2.04,Census tract,Data for this place represent multiple merged ...
48,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000205,Census Tract 2.05,Census tract,Data for this place represent multiple merged ...
49,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000206,Census Tract 2.06,Census tract,Data for this place represent multiple merged ...
77,AK22,Arctic Village,Vashrąįį K'ǫǫ,Alaska,US,68.1269,-145.538,community,1600000US0203990,Arctic Village CDP,Census designated place,Data represent information from nearest [COLUM...
166,AK130,Fort Yukon,Gwichyaa Zheh,Alaska,US,66.5647,-145.274,community,1600000US0226760,Fort Yukon city,Incorporated place,Data represent information from nearest [COLUM...


In [3]:
# use the individual fetch functions get data for GVV IDs in the test lookup table
# follow the printed URLs to QC individual values from the results table
for gvv_id in list(geoid_lu_df_test.id.unique()):
    dhc = fetch_census_data_and_compute("dhc", gvv_id, geoid_lu_df_test, print_url=True)
    print(dhc)
    acs5 = fetch_census_data_and_compute("acs5", gvv_id, geoid_lu_df_test, print_url=True)
    print(acs5)
    cdc = fetch_cdc_data_and_compute(gvv_id, geoid_lu_df_test, print_url=True)
    print(cdc)


Requesting US Census data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
  GEOID  total_population  pct_65_plus  pct_under_18
0   090           95655.0        11.46         23.95
Requesting US Census data from: https://api.census.gov/data/2020/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
   pct_w_disability  moe_pct_w_disability  pct_insured  moe_pct_insured  \
0              11.1                   1.0         90.6              1.0   

   pct_uninsured  moe_pct_uninsured GEOID  
0            9.4                1.0   090  
Requesting CDC data from: https://data.cdc.gov/reso

In [4]:
# also test the fetch_and_merge() function, which will add the comments from the comment dict
test_results = []
test_comment_dict = create_comment_dict(geoid_lu_df_test)

for gvv_id in list(geoid_lu_df_test.id.unique()):
    test_results.append(fetch_and_merge(geoid_lu_df_test, gvv_id, test_comment_dict))
test_results_df = pd.concat(test_results)

# view just the geographic info and comments
pd.options.display.max_colwidth = 999
test_results_df[['id','name','areatype','placename','comment']]

,id,name,areatype,placename,comment
0,AK124,Fairbanks,County,Fairbanks North Star Borough,"Data represent information from nearest county (Fairbanks North Star Borough), which includes Fairbanks."
0,AK103,Eagle River,Census tract,Census Tract 2.01,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
1,AK103,Eagle River,Census tract,Census Tract 2.02,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
2,AK103,Eagle River,Census tract,Census Tract 2.04,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
3,AK103,Eagle River,Census tract,Census Tract 2.05,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
4,AK103,Eagle River,Census tract,Census Tract 2.06,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
0,AK22,Arctic Village,Census designated place,Arctic Village CDP,"Data represent information from nearest census designated place (Arctic Village CDP), which includes Arctic Village."
0,AK130,Fort Yukon,Incorporated place,Fort Yukon city,"Data represent information from nearest incorporated place (Fort Yukon city), which includes Fort Yukon."


In [2]:
# if test results look good, run the fetch and merge with the full lookup table
# even with multiprocessing, this might take a while to complete the many calls to the various APIs
# heavy API usage may be "throttled" by the service ... ~ 5-6 minutes seemed to be the norm here
results_df = run_fetch_and_merge(geoid_lu_df)
# then aggregate any rows with duplicate IDs
aggregated_results_df = aggregate_results(results_df)

No response, check your URL
Aggregating values for AK103: Eagle River
Aggregating values for AK439: Joint Base Elmendorf-Richardson


/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df[col] = sub_df['total_population'] * sub_df[col] / 100
/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df[col] = sub_df['total_population'] * sub_df[col] / 100


In [3]:
# compare the original one-to-many results with aggregated results
dups = results_df[results_df.duplicated(subset='id')]['id'].unique().tolist()
results_df[results_df['id'].isin(dups)]

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_hispanic_latino,pct_white,...,pct_copd,pct_hd,pct_stroke,pct_diabetes,pct_kd,pct_minority,pct_no_hsdiploma,pct_below_150pov,pct_no_bband,comment
0,AK103,Eagle River,Census tract,Census Tract 2.01,020000201,4318.0,7.74,26.98,9.59,66.23,...,4.3,3.0,1.7,5.2,1.8,26.9,7.1,11.5,8.9,Data for this place represent multiple merged ...
1,AK103,Eagle River,Census tract,Census Tract 2.02,020000202,6384.0,12.97,25.50,7.50,73.86,...,4.7,3.7,2.1,6.0,2.1,28.2,3.9,6.9,4.7,Data for this place represent multiple merged ...
2,AK103,Eagle River,Census tract,Census Tract 2.04,020000204,3582.0,12.65,21.30,5.30,76.44,...,4.2,3.3,1.8,5.6,2.0,19.7,1.9,6.9,3.3,Data for this place represent multiple merged ...
3,AK103,Eagle River,Census tract,Census Tract 2.05,020000205,7421.0,7.16,30.64,9.03,73.22,...,NaN,NaN,NaN,NaN,NaN,19.1,0.8,4.0,1.5,Data for this place represent multiple merged ...
4,AK103,Eagle River,Census tract,Census Tract 2.06,020000206,3413.0,13.45,28.13,6.36,75.71,...,NaN,NaN,NaN,NaN,NaN,11.4,0.9,2.6,3.5,Data for this place represent multiple merged ...
0,AK439,Joint Base Elmendorf-Richardson,Census tract,Census Tract 9801,020980100,4592.0,0.68,25.07,16.86,59.82,...,NaN,NaN,NaN,NaN,NaN,45.9,1.6,7.9,0.7,Data for this place represent multiple merged ...
1,AK439,Joint Base Elmendorf-Richardson,Census tract,Census Tract 9802,020980200,6725.0,2.13,25.25,17.67,57.44,...,NaN,NaN,NaN,NaN,NaN,44.8,2.5,19.6,7.1,Data for this place represent multiple merged ...


In [4]:
aggregated_results_df[aggregated_results_df['id'].isin(dups)]

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_hispanic_latino,pct_white,...,pct_copd,pct_hd,pct_stroke,pct_diabetes,pct_kd,pct_minority,pct_no_hsdiploma,pct_below_150pov,pct_no_bband,comment
421,AK103,Eagle River,Census tract,"Census Tract 2.01, Census Tract 2.02, Census T...","020000201, 020000202, 020000204, 020000205, 02...",25118.0,10.37,27.03,7.84,72.98,...,NaN,NaN,NaN,NaN,NaN,21.79,2.84,6.25,4.11,Data for this place represent multiple merged ...
422,AK439,Joint Base Elmendorf-Richardson,Census tract,"Census Tract 9801, Census Tract 9802","020980100, 020980200",11317.0,1.54,25.18,17.34,58.41,...,NaN,NaN,NaN,NaN,NaN,45.25,2.13,14.85,4.50,Data for this place represent multiple merged ...


In [5]:
# save to CSV
aggregated_results_df.to_csv('tbl/data_to_export.csv', index=False)